# 복합 토픽 모델링(Combined Topic Modeling)

이 튜토리얼에서는 복합 토픽 모델(**Combined Topic Model**)을 사용하여 문서의 집합에서 토픽을 추출해보겠습니다.

## 토픽 모델(Topic Models)

토픽 모델을 사용하면 비지도 학습 방식으로 문서에 잠재된 토픽을 추출할 수 있습니다.

## 문맥을 반영한 토픽 모델(Contextualized Topic Models)
문맥을 반영한 토픽 모델(Contextualized Topic Models, CTM)이란 무엇일까요? CTM은 BERT 임베딩의 표현력과 토픽 모델의 비지도 학습의 능력을 결합하여 문서에서 주제를 가져오는 토픽 모델의 일종입니다.

# Contextualized Topic Models, CTM 설치

contextualized topic model 라이브러리를 설치합시다.

In [1]:
!pip install contextualized-topic-models==2.2.0

In [2]:
!pip install pyldavis

In [3]:
!pip install eunjeon

In [8]:
from eunjeon import Mecab
mecab = Mecab()

print(mecab.pos("품사 태깅을 지원합니다."))
print(mecab.morphs("형태소 분리를 지원합니다."))
print(mecab.nouns("명사에 해당하는 형태소만 추출합니다."))

[('품사', 'NNG'), ('태', 'NNG'), ('깅을', 'UNKNOWN'), ('지원', 'NNG'), ('합니다', 'XSV+EF'), ('.', 'SF')]
['형태소', '분리', '를', '지원', '합니다', '.']
['명사', '해당', '형태소', '추출']


## 노트북 재시작

원활한 실습을 위해서 노트북을 재시작 할 필요가 있습니다.

상단에서 런타임 > 런타임 재시작을 클릭해주세요.

# 데이터

학습을 위한 데이터가 필요합니다. 여기서는 하나의 라인(line)에 하나의 문서로 구성된 파일이 필요한데요. 우선, 여러분들의 데이터가 없다면 여기서 준비한 파일로 실습을 해봅시다.

In [9]:
!python -m wget https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt


Saved under 2016-10-20 (4).txt


In [10]:
text_file = "2016-10-20.txt"

# 필요한 것들을 임포트

In [11]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from eunjeon import Mecab
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

## 전처리

In [12]:
documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]

In [13]:
documents[:5]

['',
 '19  1990  52 1 22',
 '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽

In [14]:
not '19  1990  52 1 22'.replace(' ', '').isdecimal()

False

In [15]:
preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

100%|████████████████████████████████████████████| 30091/30091 [00:00<00:00, 248786.83it/s]


In [16]:
len(preprocessed_documents)

27540

In [17]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [18]:
custom_tokenizer = CustomTokenizer(Mecab())

In [19]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [20]:
train_bow_embeddings = vectorizer.fit_transform(preprocessed_documents)

In [21]:
print(train_bow_embeddings.shape)

(27540, 3000)


In [22]:
print(train_bow_embeddings[0])

  (0, 2732)	2
  (0, 2665)	5
  (0, 2533)	1
  (0, 1965)	4
  (0, 229)	3
  (0, 1365)	7
  (0, 1777)	5
  (0, 266)	11
  (0, 350)	1
  (0, 19)	4
  (0, 1820)	9
  (0, 162)	4
  (0, 1817)	4
  (0, 2039)	6
  (0, 1721)	13
  (0, 1307)	9
  (0, 2535)	9
  (0, 1076)	5
  (0, 1321)	2
  (0, 1852)	3
  (0, 695)	1
  (0, 238)	1
  (0, 522)	2
  (0, 2477)	1
  (0, 1341)	2
  :	:
  (0, 1354)	1
  (0, 2532)	1
  (0, 1482)	2
  (0, 1277)	1
  (0, 843)	1
  (0, 640)	1
  (0, 2435)	1
  (0, 968)	1
  (0, 2011)	1
  (0, 653)	1
  (0, 1337)	1
  (0, 2523)	1
  (0, 2439)	1
  (0, 868)	1
  (0, 1684)	1
  (0, 2218)	1
  (0, 913)	1
  (0, 1308)	1
  (0, 16)	1
  (0, 1662)	1
  (0, 1603)	1
  (0, 2768)	1
  (0, 2843)	1
  (0, 2771)	1
  (0, 1071)	2


In [25]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

In [26]:
len(vocab)

3000

In [27]:
train_contextualized_embeddings = bert_embeddings_from_list(preprocessed_documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Batches:   0%|          | 0/138 [00:00<?, ?it/s]

In [28]:
print(train_contextualized_embeddings.shape)

(27540, 768)


In [29]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

In [45]:
print(training_dataset.idx2token)
print(training_dataset.X_bow)
print(training_dataset.X_contextual.shape)
# dir(training_dataset)

{0: '00', 1: '000', 2: '01', 3: '02', 4: '10', 5: '100', 6: '1000', 7: '101', 8: '11', 9: '119', 10: '12', 11: '120', 12: '13', 13: '14', 14: '15', 15: '150', 16: '16', 17: '17', 18: '18', 19: '19', 20: '20', 21: '200', 22: '2000', 23: '2005', 24: '2006', 25: '2007', 26: '2008', 27: '2009', 28: '2010', 29: '2011', 30: '2012', 31: '2013', 32: '2014', 33: '2015', 34: '2016', 35: '2017', 36: '2018', 37: '2019', 38: '2020', 39: '21', 40: '22', 41: '23', 42: '24', 43: '25', 44: '26', 45: '27', 46: '28', 47: '29', 48: '30', 49: '300', 50: '3000', 51: '31', 52: '32', 53: '33', 54: '34', 55: '35', 56: '36', 57: '37', 58: '38', 59: '39', 60: '40', 61: '400', 62: '4000', 63: '41', 64: '42', 65: '43', 66: '44', 67: '45', 68: '46', 69: '47', 70: '48', 71: '49', 72: '50', 73: '500', 74: '5000', 75: '51', 76: '52', 77: '53', 78: '54', 79: '55', 80: '57', 81: '58', 82: '59', 83: '60', 84: '600', 85: '6000', 86: '61', 87: '62', 88: '63', 89: '64', 90: '65', 91: '66', 92: '67', 93: '68', 94: '70', 95: 

## Combined TM 학습하기
이제 토픽 모델을 학습합니다. 여기서는 하이퍼파라미터에 해당하는 토픽의 개수(n_components)로는 50개를 선정합니다.

In [46]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=50, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [550800/550800]	Train Loss: 853.878824160596	Time: 0:00:20.711500: : 20it [06:57, 20.87s/it] 


# 토픽들

학습 후에는 토픽 모델이 선정한 토픽들을 보려면 아래의 메소드를 사용합니다.

```
get_topic_lists
```
해당 메소드에는 각 토픽마다 몇 개의 단어를 보고 싶은지에 해당하는 파라미터를 넣어즐 수 있습니다.

In [49]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['영상', '뉴시스', '언론', '공감', '사진'],
             1: ['그룹', '공개', '싱글', '앨범', '멤버'],
             2: ['그램', '화보', '공개', '출연', '조안'],
             3: ['으로', '에서', '습니다', '습니까', '말씀'],
             4: ['사랑', '남자', '우리', '제작', '때문'],
             5: ['정산', '연말', '공제', '신용', '미리'],
             6: ['영상', '뉴시스', '사진', '언론', '공감'],
             7: ['모습', '화신', '방송', '남지현', '질투'],
             8: ['으로', '최근', '2016', '에서', '매물'],
             9: ['따로', '된다는', '부여', '바뀌', '이탈리아'],
             10: ['중구', '디자인', '컬렉션', '2017', '열린'],
             11: ['지상파', '가이드라인', '협상', '방송', '유료'],
             12: ['서현진', '사랑', '낭만', '닥터', '박보검'],
             13: ['제작', '타임', '하늘', '남자', '스퀘어'],
             14: ['습니다', '도어', '기관사', '전동차', '출입문'],
             15: ['코리아', '뉴스', '모임', '충분', '여의도'],
             16: ['서울대', '캠퍼스', '교육', '연구', '학생'],
             17: ['부터', '가을', '까지', '여행', '한다'],
             18: ['트럼프', '라고', '성추행', '도널드', '클린턴'],
             19: ['연방', '시장'

In [50]:
ctm.get_topics(10)

defaultdict(list,
            {0: ['영상', '뉴시스', '언론', '공감', '사진', '제보', '가치', '여러분', '02', '판단'],
             1: ['그룹', '공개', '싱글', '앨범', '멤버', '엔터', '수록', '데뷔', '아이오', '발매'],
             2: ['그램', '화보', '공개', '출연', '조안', '스타', '한편', '웨딩', '모습', '게재'],
             3: ['으로',
              '에서',
              '습니다',
              '습니까',
              '말씀',
              '그래서',
              '다고',
              '그렇',
              '거든요',
              '까지'],
             4: ['사랑', '남자', '우리', '제작', '때문', '김유정', '차태현', '배우', '그린', '상실'],
             5: ['정산', '연말', '공제', '신용', '미리', '소득', '카드', '까지', '으로', '국세청'],
             6: ['영상', '뉴시스', '사진', '언론', '공감', '제보', '가치', '여러분', '02', '화제'],
             7: ['모습',
              '화신',
              '방송',
              '남지현',
              '질투',
              '출연',
              '김국진',
              '매주',
              '공효진',
              '조정석'],
             8: ['으로',
              '최근',
              '2016',
              '에서',
     

# 시각화

우리의 토픽들을 시각화하기 위해서는 PyLDAvis를 사용합니다.

위에서 출력한 토픽 번호는 pyLDAvis에서 할당한 토픽 번호와 일치하지 않으므로 주의합시다.  
가령, 48번 토픽이었던 ['원유', '유가', '뉴욕', '오른', '연방', '마쳤', '서부', '달러', '51', '지수']가 아래의 PyLDAvis에서는 24번 토픽이 되었습니다.


In [51]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

Sampling: [10/10]: : 10it [03:22, 20.28s/it]


참고 자료 : https://github.com/MilaNLProc/contextualized-topic-models